In [1]:
import os
import pandas as pd
import psycopg2
from dotenv import load_dotenv

load_dotenv()

True

In [53]:
# Establish a connection to the database
# Replace 'database_name', 'user', and 'password' with your database details
conn = psycopg2.connect(
    dbname='dsn', 
    user=os.getenv('USERNAME_DB'),
    password=os.getenv('PASSWORD_DB'), 
    host='10.0.0.1'
)

# Create a cursor object
cur = conn.cursor()

# Execute a query
cur.execute("""
    select
	EXTRACT(MONTH FROM ddadtsalaire_base.date_debut_paie) as MONTH,
	EXTRACT(YEAR FROM ddadtsalaire_base.date_debut_paie) as YEAR,
  ddadtassure.id,
	ddadtcontrat.num_contrat,
  ddadtassure.sexe,
  EXTRACT(YEAR FROM age(NOW(), TO_DATE(ddadtassure.date_naissance, 'DDMMYYYY'))) as age,
  ddadtcontrat.unite_mesure,
  ddadtcontrat.quotite,
  ddadtcontrat.quotite_categorie,
  ddadtcontrat.pcs_ese,
	ddadtcontrat.statut_conventionnel, -- CSP
	ddadtcontrat.dispositif_politique, -- type contrat (pour exclusions)
  ddadtcontrat.nature_contrat, -- S21.G00.40.007
  ddadtsalaire_base.montant as SALAIRE_BRUT
  from  ddadtsalaire_base
	left join ddadtversement on ddadtversement.id=ddadtsalaire_base.id_versement
  left join ddadtemployeur_assure on ddadtemployeur_assure.id=ddadtversement.id_employeur_assure
  left join ddadtassure on ddadtassure.id=ddadtemployeur_assure.id_assure
  left join ddadtcontrat on ddadtcontrat.id_employeur_assure=ddadtemployeur_assure.id and ddadtsalaire_base.numero_contrat=ddadtcontrat.num_contrat
  where ddadtemployeur_assure.id_employeur=1069987;""")

# Fetch the results
records = cur.fetchall()

df_salaires = pd.DataFrame(records, columns=['month', 'year', 'id_assure', 'num_contrat', 'sexe', 'age', 'unite_mesure',
       'quotite', 'quotite_categorie', 'pcs_ese', 'statut_conventionnel', 'dispositif_politique',
       'nature_contrat', 'salaire_brut'])

# tous les congés maternité qui ont repris en 2023
cur.execute("""

select
	ddadtemployeur_assure.id_assure,
  id_contrat,motif_arret,
  date_dernier_jour_travaille,
  max(date_reprise) as MAX_REPRISE,

	-- salaire au dernier jour travaillé
  (
    SELECT SUM(montant)
    from  ddadtsalaire_base
    left join ddadtversement on ddadtversement.id=ddadtsalaire_base.id_versement
    left join ddadtemployeur_assure ddadtemployeur_assure2  on ddadtemployeur_assure2.id=ddadtversement.id_employeur_assure
    where ddadtemployeur_assure2.id_employeur=1069987 and  ddadtemployeur_assure2.id_assure=ddadtemployeur_assure.id_assure and EXTRACT(MONTH from date_debut_paie)=EXTRACT(MONTH from max(date_dernier_jour_travaille)) and EXTRACT(YEAR from date_debut_paie)=EXTRACT(YEAR from max(date_dernier_jour_travaille))
  ) as SALAIRE1,

  -- salaire à la reprise
  (
    SELECT SUM(montant)
    from  ddadtsalaire_base
    left join ddadtversement on ddadtversement.id=ddadtsalaire_base.id_versement
    left join ddadtemployeur_assure ddadtemployeur_assure2  on ddadtemployeur_assure2.id=ddadtversement.id_employeur_assure
    where ddadtemployeur_assure2.id_employeur=1069987 and  ddadtemployeur_assure2.id_assure=ddadtemployeur_assure.id_assure and EXTRACT(MONTH from date_debut_paie)=EXTRACT(MONTH from max(date_reprise)) and EXTRACT(YEAR from date_debut_paie)=EXTRACT(YEAR from max(date_reprise))
  ) as SALAIRE2


 from ddadtarret_trav
left join ddadtcontrat on ddadtcontrat.id=ddadtarret_trav.id_contrat
left join ddadtemployeur_assure on ddadtemployeur_assure.id=ddadtcontrat.id_employeur_assure
where ddadtemployeur_assure.id_employeur=1069987 and motif_arret='02' and EXTRACT(YEAR from date_reprise)=2023
group by ddadtemployeur_assure.id_assure,id_contrat,motif_arret,date_dernier_jour_travaille

""")

records = cur.fetchall()

df_maternites = pd.DataFrame(records, columns=["id_assure", "id_contrat", "motif_arret", "date_dernier_jour_travaille", "max_reprise", "salaire1", "salaire2"])

# Close the cursor and connection
cur.close()
conn.close()

In [61]:
json_structure[f"index . maternité . nombre total"] = len(df_maternites)
json_structure[f"index . maternité . nombre augmentés"] = (df_maternites["salaire1"]<df_maternites["salaire2"]).sum()
json_structure

{'index . écart rémunérations . employés . 50 ans et plus . remunération annuelle brute moyenne par EQTP . femmes': 1865.7451153846152,
 'index . écart rémunérations . employés . 50 ans et plus . nombre salariés . femmes': 79.0,
 'index . écart rémunérations . ingénieurs et cadres . 50 ans et plus . remunération annuelle brute moyenne par EQTP . femmes': 3726.8087255527216,
 'index . écart rémunérations . ingénieurs et cadres . 50 ans et plus . nombre salariés . femmes': 168.0,
 'index . écart rémunérations . techniciens et agents de maitrise . 50 ans et plus . remunération annuelle brute moyenne par EQTP . femmes': 2413.463118131868,
 'index . écart rémunérations . techniciens et agents de maitrise . 50 ans et plus . nombre salariés . femmes': 112.0,
 'index . écart rémunérations . employés . de 30 à 39 ans . remunération annuelle brute moyenne par EQTP . femmes': 1785.351361172161,
 'index . écart rémunérations . employés . de 30 à 39 ans . nombre salariés . femmes': 20.0,
 'index . 

### Filtering data

In [3]:
# Calcul du taux ETP
df_salaires["equiv_etp"] = df_salaires["quotite"] / df_salaires["quotite_categorie"]
df_salaires["equiv_etp"].apply(lambda x: min(x, 1))

# Selection des personnes ayant un contrat de bonne nature légale, exclusion des apprentis et des contrats pro
etp_annee = df_salaires.loc[(~df_salaires["nature_contrat"].isin(["08", "20", "21", "29", "50", "51","52", "54", "70", "81", "90"]))
                            & (~df_salaires["dispositif_politique"].isin(["61", "64", "65"]))
                            & (df_salaires["year"] == 2022), ["id_assure", "equiv_etp"]].groupby(by="id_assure").sum()
personnes_eligibles = etp_annee[etp_annee["equiv_etp"] >= 6].index.values

df_salaires_filtered = df_salaires[(df_salaires["year"] == 2022) & (df_salaires["id_assure"].isin(personnes_eligibles))].copy()

### Mapping 

In [4]:
# Fais le mapping CSP
mapping = {
    "01": 'ouvriers',
    "02": 'ouvriers',
    "03": 'ingénieurs et cadres',
    "04": 'ingénieurs et cadres',
    "05": 'techniciens et agents de maitrise',
    "06": 'employés',
    "07": 'ouvriers',
}
df_salaires_filtered["categorie_csp"] = df_salaires_filtered["statut_conventionnel"].map(mapping)

def categorize_age(age):
    if age < 30:
        return 'moins de 30 ans'
    elif 30 <= age < 40:
        return 'de 30 à 39 ans'
    elif 40 <= age < 50:
        return 'de 40 à 49 ans'
    else:
        return '50 ans et plus'

# Apply the function to create the AgeCategory column
df_salaires_filtered['categorie_age'] = df_salaires_filtered['age'].apply(categorize_age)

### Indicateur 1 (rémunérations)

In [5]:
import json

df_salaires_filtered["salaire_proratise"] = df_salaires_filtered["salaire_brut"] * df_salaires_filtered["equiv_etp"]
df = df_salaires_filtered.groupby(by=["id_assure", "sexe", "categorie_age", "categorie_csp"], as_index=False).aggregate({"salaire_proratise":"sum", "equiv_etp":"sum"})
df["salaire_moyen"] = df["salaire_proratise"] / df["equiv_etp"]

# Attention on peut avoir des problèmes si plusieurs contrat pour la même personne.  Sinon on peut faire le calcul du salaire EQTP par salariés sur l'année. Calcul ETP aussi faux
df = df.groupby(by=["sexe", "categorie_age", "categorie_csp"]).aggregate({"salaire_moyen": "mean", "id_assure": "count"})
json_structure = {}

for (sexe, age, csp), row in df.iterrows() :
    key_base = f"index . écart rémunérations . {csp} . {age}"
    
    if sexe == "H":
        sex_key = "hommes"
    else:
        sex_key = "femmes"
        
    json_structure[f"{key_base} . remunération annuelle brute moyenne par EQTP . {sex_key}"] = row['salaire_moyen']
    json_structure[f"{key_base} . nombre salariés . {sex_key}"] = row['id_assure']


# Convert to JSON
json_output = json.dumps(json_structure, indent=4, ensure_ascii=False)

### Indicateur 5 (hautes rémunérations)

In [20]:
# extraire top salaire par personne sur l'année, selectionner le top 10

In [38]:
df_salaires_filtered["salaire_proratise"] = df_salaires_filtered["salaire_brut"] * df_salaires_filtered["equiv_etp"]
df = df_salaires_filtered.groupby(by=["id_assure", "sexe", "month", "year"], as_index=False).agg({"equiv_etp":"sum", "salaire_proratise":"sum"})
df["salaire_mensuel"] = df["salaire_proratise"] / df["equiv_etp"].replace(0, 1)
df_rem = df.groupby(by=["sexe","id_assure"], as_index=False).agg({"salaire_mensuel":"max"})
df_rem = df_rem.sort_values(by="salaire_mensuel", ascending=False)
repartition = df_rem.iloc[:10]["sexe"].value_counts()
json_structure[f"index . hautes rémunérations . femmes"] = repartition.get("F")
json_structure[f"index . hautes rémunérations . hommes"] = repartition.get("H")

### Indicateur 4 (maternité)
